In [1]:
!pip install antigranular

In [2]:
import antigranular as ag
session = ag.login(<client_id>,<client_secret>, dataset = "Car Evaluation")

Connected to Antigranular server session id: 14b3b0e1-a3a5-4d8b-bd2d-a3548d4db90f
Cell magic '%%ag' registered successfully, use `%%ag` in a notebook cell to execute your python code on Antigranular private python server


### Loading the dataset

In [3]:
%%ag

from op_pandas import PrivateDataFrame , PrivateSeries
from ag_utils import load_dataset

# Obtaining the dictionary containing private objects
response = load_dataset("Car Evaluation")

# Unpacking the response dictionary
train_x = response["train_x"]
train_y = response["train_y"]
test_x = response["test_x"]

### Encoding the categories

In [4]:
%%ag
def fun(x):
  arr = ['vhigh','high','med','low']
  for i in range(len(arr)):
    if arr[i] == x:
      return i

train_x['buying'] = train_x['buying'].map(fun, output_bounds = (0,3))
train_x['maint'] = train_x['maint'].map(fun, output_bounds = (0,3))

test_x['buying'] = test_x['buying'].map(fun)
test_x['maint'] = test_x['maint'].map(fun)

In [5]:
%%ag
def fun(x):
  arr = ['2','3','4','5more']
  for i in range(len(arr)):
    if arr[i] == x:
      return i

train_x['doors'] = train_x['doors'].map(fun, output_bounds = (0,3))

test_x['doors'] = test_x['doors'].map(fun)

In [6]:
%%ag
def fun(x):
  arr = ['2','4','more']
  for i in range(len(arr)):
    if arr[i] == x:
      return i

train_x['persons'] = train_x['persons'].map(fun, output_bounds = (0,2))

test_x['persons'] = test_x['persons'].map(fun)

In [7]:
%%ag
def fun(x):
  arr = ['small','med','big']
  for i in range(len(arr)):
    if arr[i] == x:
      return i

train_x['lug_boot'] = train_x['lug_boot'].map(fun, output_bounds = (0,2))

test_x['lug_boot'] = test_x['lug_boot'].map(fun)

In [8]:
%%ag
def fun(x):
  arr = ['low','med','high']
  for i in range(len(arr)):
    if arr[i] == x:
      return i

train_x['safety'] = train_x['safety'].map(fun, output_bounds = (0,2))

test_x['safety'] = test_x['safety'].map(fun)

### Describe

In [9]:
%%ag
from ag_utils import export
export(train_x.describe(eps=1), 'describe')

Total epsilon spent = 1.0000

Setting up exported variable in local environment: describe


In [10]:
describe

,buying,maint,doors,persons,lug_boot,safety
count,1345.000000,1345.000000,1345.000000,1345.000000,1345.000000,1345.000000
mean,1.523040,1.538574,1.562430,1.181534,0.871113,0.929916
std,1.364976,1.393181,0.362956,0.262635,0.701924,0.908569
min,0.232517,0.467182,0.443814,0.383916,0.488454,0.387402
25%,0.140545,0.651546,0.905304,0.188176,0.723930,0.880495
50%,1.941969,1.909901,1.134066,1.942191,0.023650,0.209483
75%,2.286090,2.745749,2.632030,1.932848,1.995699,1.383640
max,2.839239,2.269671,2.590038,1.021235,1.238869,1.259505


In [11]:
%%ag
meta = train_x.metadata
export(meta, "metadata")

Setting up exported variable in local environment: metadata


In [12]:
metadata

{'buying': (0, 3),
 'maint': (0, 3),
 'doors': (0, 3),
 'persons': (0, 2),
 'lug_boot': (0, 2),
 'safety': (0, 2)}

### Random Forest Classifier

In [13]:
%%ag
from op_diffprivlib.models import RandomForestClassifier

# creating the bounds for the RandomForest
l_meta, r_meta = [], []
for l,r in meta.values():
  l_meta.append(l)
  r_meta.append(r)

In [14]:
%%ag
# fitting the classifier
random_forest = RandomForestClassifier(n_estimators = 100, epsilon = 6, bounds = (l_meta, r_meta), classes = ['unacc','acc','good','vgood'])
random_forest.fit(train_x, train_y)

/usr/local/lib/python3.10/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



In [15]:
%%ag
# prediction
pred = random_forest.predict(test_x)
export(pred, "pred")

Setting up exported variable in local environment: pred


In [16]:
import pandas as pd
# checking the metrics
y_df = pd.DataFrame(pred)
result = session.submit_predictions(y_df)
print(result)

{'score': {'leaderboard': 0.23099661905938396, 'logs': {'BIN_ACC': 0.23099661905938396}}}


In [17]:
# Privacy budget spent
session.privacy_odometer()

{'total_epsilon_used': 7.000000000000001, 'total_delta_used': 0.0}